# 5. Text Detection, Recognition & Spotting on the CCPD 2019 dataset


## Mount GDrive


In [1]:
from google.colab import drive

drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd "/content/drive/My Drive"
%cd "CCPD2019"
!ls

/content/drive/My Drive
/content/drive/My Drive/CCPD2019
CCPD2019.zip  train_crop     train.txt	val_crop     val.txt
train	      train_rec.txt  val	val_rec.txt


## Install PPOCR


In [3]:
%cd "/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/"
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd ./PaddleOCR
!pip install -r requirements.txt
!pip install paddlepaddle
!pip install levenshtein

/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition
fatal: destination path 'PaddleOCR' already exists and is not an empty directory.
/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/PaddleOCR
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-4.23.0-cp37-abi3-manylinux2014_x86_64.whl (304 kB)
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlepaddle 2.4.2 requires protobuf<=3.20.0,>=3.1.0, but you have protobuf 3.20.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.9.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery-storage 2.19.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Uncomment them to download checkpoints for east and crnn to your google drive.

%cd "/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_east"
!rm -rf *.tar*
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_mv3_east_v2.0_train.tar
!tar xvf det_mv3_east_v2.0_train.tar

%cd "/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_crnn"
!rm -rf *.tar*
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/rec_mv3_none_bilstm_ctc_v2.0_train.tar
!tar xvf rec_mv3_none_bilstm_ctc_v2.0_train.tar


/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_east
--2023-05-11 10:04:52--  https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_mv3_east_v2.0_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33679360 (32M) [application/x-tar]
Saving to: ‘det_mv3_east_v2.0_train.tar’

det_mv3_east_v2.0_t 100%[===================>]  32.12M  8.44MB/s    in 20s     

2023-05-11 10:05:14 (1.60 MB/s) - ‘det_mv3_east_v2.0_train.tar’ saved [33679360/33679360]

det_mv3_east_v2.0_train/
det_mv3_east_v2.0_train/best_accuracy.pdparams
det_mv3_east_v2.0_train/best_accuracy.states
det_mv3_east_v2.0_train/best_accuracy.pdopt
det_mv3_east_v2.0_train/train.log
/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_crnn
--2023-05-11 10:

## Define Commonly Used Functions


In [5]:
%cd "/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/PaddleOCR"
!ls

/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/PaddleOCR
applications  doc	   paddleocr.py  README_ch.md	   StyleText
benchmark     __init__.py  ppocr	 README.md	   test_tipc
configs       LICENSE	   PPOCRLabel	 requirements.txt  tools
deploy	      MANIFEST.in  ppstructure	 setup.py	   train.sh


In [6]:
import os
import cv2
import glob
import sys
import json
import yaml
import shutil
import numpy as np
from tqdm import tqdm
from collections import defaultdict

import paddle
from paddle import fluid
from ppocr.data import create_operators, transform
from ppocr.modeling.architectures import build_model
from ppocr.postprocess import build_post_process
from ppocr.utils.save_load import load_model
from ppocr.utils.utility import get_image_file_list

__dir__ = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/PaddleOCR"
)
sys.path.append(__dir__)
sys.path.append(os.path.abspath(os.path.join(__dir__, "..")))

from det_eval import evaluation
from collections import defaultdict
from rec_eval import total_accuracy, total_edit_distance


# This is to reset dygraph
def reset_dygraph():
    fluid.dygraph.disable_dygraph()
    fluid.dygraph.enable_dygraph()


# HMean for text detection and spotting
def det_eval(gt_path, det_path, eval_config):
    # Prepare GT
    gt_dict = defaultdict(list)
    with open(gt_path, mode="r") as in_txt:
        lines = in_txt.readlines()
        for line in lines:
            line = line.strip()
            values = line.split("\t")
            img_id = values[0]
            annos = json.loads(values[1])

            for anno in annos:
                trans = anno["transcription"]
                bbox = anno["points"]
                xs = [x[0] for x in bbox]
                ys = [x[1] for x in bbox]
                xmin = min(xs)
                xmax = max(xs)
                ymin = min(ys)
                ymax = max(ys)
                gt_dict[img_id].append([xmin, ymin, xmax, ymax, trans])

    # Prepare Det
    det_dict = defaultdict(list)
    with open(det_path, mode="r") as in_txt:
        lines = in_txt.readlines()
        for line in lines:
            line = line.strip()
            values = line.split("\t")
            img_id = values[0]
            annos = json.loads(values[1])

            for anno in annos:
                bbox = anno["points"]
                xs = [x[0] for x in bbox]
                ys = [x[1] for x in bbox]
                xmin = min(xs)
                xmax = max(xs)
                ymin = min(ys)
                ymax = max(ys)

                width = xmax - xmin
                height = ymax - ymin

                pred_entry = [xmin, ymin, xmax, ymax]

                if eval_config["WORD_SPOTTING"]:
                    trans = anno["transcription"]
                    pred_entry.append(trans)

                det_dict[img_id].append(pred_entry)

    resDict = evaluation(gt_dict, det_dict, eval_config)
    return resDict


# Accuracy and Edit Distance for text recognition - data preparation
def data_prep_rec_eval(gt_path, det_path):
    # Prepare GT
    gt_dict = {}
    with open(gt_path, mode="r") as in_txt:
        lines = in_txt.readlines()
        for line in lines:
            line = line.strip()
            values = line.split("\t")
            img_id = values[0]
            gt_dict[img_id] = values[1]

    # Prepare Det
    det_dict = {}
    with open(det_path, mode="r") as in_txt:
        lines = in_txt.readlines()
        for line in lines:
            line = line.strip()
            values = line.split("\t")
            img_id = values[0]
            det_dict[img_id] = values[1]

    gt = []
    pred = []

    for key, val in gt_dict.items():
        if key in det_dict.keys():
            gt.append(val)
            pred.append(det_dict[key])

    return gt, pred


# Draw detection result from PaddleOCR
def draw_det_res(dt_boxes, img, img_name, save_path):
    if len(dt_boxes) > 0:
        src_im = img
        for box in dt_boxes:
            box = np.array(box).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(src_im, [box], True, color=(255, 255, 0), thickness=2)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        save_path = os.path.join(save_path, os.path.basename(img_name))
        cv2.imwrite(save_path, src_im)


# Load PaddleOCR config
def load_config(file_path):
    """
    Load config from yml/yaml file.
    Args:
        file_path (str): Path of the config file to be loaded.
    Returns: global config
    """
    _, ext = os.path.splitext(file_path)
    assert ext in [".yml", ".yaml"], "only support yaml files for now"
    config = yaml.load(open(file_path, "rb"), Loader=yaml.Loader)
    return config


# Init detection model of PaddleOCR
def init_det(config_path, ft_model_path=None):
    config = load_config(config_path)
    global_config = config["Global"]
    if ft_model_path != None:
        global_config["pretrained_model"] = ft_model_path

    # build model
    model = build_model(config["Architecture"])

    load_model(config, model)
    # build post process
    post_process_class = build_post_process(config["PostProcess"])

    # create data ops
    transforms = []
    for op in config["Eval"]["dataset"]["transforms"]:
        op_name = list(op)[0]
        if "Label" in op_name:
            continue
        elif op_name == "KeepKeys":
            op[op_name]["keep_keys"] = ["image", "shape"]
        transforms.append(op)

    ops = create_operators(transforms, global_config)

    model.eval()
    return config, model, ops, post_process_class


# Carry out text detection using PaddleOCR
def det_ppocr(out_path, vis_path, det_yml, img_path, out_txt_name, ft_model_path=None):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    if not os.path.exists(vis_path):
        os.makedirs(vis_path)

    # Paddle OCR Configs
    detConfig, detModel, detOPS, detPost = init_det(det_yml, ft_model_path)

    images = glob.glob(os.path.join(img_path, "*"))  # load images

    with open(os.path.join(out_path, f"{out_txt_name}.txt"), mode="w") as out_f:
        for idx, img_name in enumerate(tqdm(images)):
            bbox_outputs = []
            with open(img_name, "rb") as f:
                img = f.read()
                data = {"image": img}

            batch = transform(data, detOPS)
            images = np.expand_dims(batch[0], axis=0)
            shape_list = np.expand_dims(batch[1], axis=0)
            images = paddle.to_tensor(images)

            # forward & post process
            preds = detModel(images)
            det_res = detPost(preds, shape_list)

            # parser boxes if post_result is dict
            if isinstance(det_res, dict):
                for k in det_res.keys():
                    boxes = det_res[k][0]["points"]
            else:
                boxes = det_res[0]["points"]

            # write predictions to images
            draw_det_res(boxes, cv2.imread(img_name), img_name, vis_path)

            boxes_len = len(boxes)
            if boxes_len == 0:
                print(
                    f"No output for {os.path.basename(img_name)}, bbox_len: {boxes_len}"
                )
                continue
            else:
                for box in boxes:
                    box_list = box.tolist()

                    current_res_dict = {"points": box_list}
                    bbox_outputs.append(current_res_dict)

            out_f.write(f"{os.path.basename(img_name)}\t{json.dumps(bbox_outputs)}\n")


# Init recognition model of PaddleOCR
def init_rec(config_path, ft_model_path=None):
    config = load_config(config_path)
    global_config = config["Global"]
    if ft_model_path != None:
        global_config["pretrained_model"] = ft_model_path

    # build post process
    post_process_class = build_post_process(config["PostProcess"], global_config)

    # build model
    if hasattr(post_process_class, "character"):
        char_num = len(getattr(post_process_class, "character"))
        if config["Architecture"]["algorithm"] in [
            "Distillation",
        ]:  # distillation model
            for key in config["Architecture"]["Models"]:
                if (
                    config["Architecture"]["Models"][key]["Head"]["name"] == "MultiHead"
                ):  # for multi head
                    out_channels_list = {}
                    if config["PostProcess"]["name"] == "DistillationSARLabelDecode":
                        char_num = char_num - 2
                    out_channels_list["CTCLabelDecode"] = char_num
                    out_channels_list["SARLabelDecode"] = char_num + 2
                    config["Architecture"]["Models"][key]["Head"][
                        "out_channels_list"
                    ] = out_channels_list
                else:
                    config["Architecture"]["Models"][key]["Head"][
                        "out_channels"
                    ] = char_num
        elif (
            config["Architecture"]["Head"]["name"] == "MultiHead"
        ):  # for multi head loss
            out_channels_list = {}
            if config["PostProcess"]["name"] == "SARLabelDecode":
                char_num = char_num - 2
            out_channels_list["CTCLabelDecode"] = char_num
            out_channels_list["SARLabelDecode"] = char_num + 2
            config["Architecture"]["Head"]["out_channels_list"] = out_channels_list
        else:  # base rec model
            config["Architecture"]["Head"]["out_channels"] = char_num

    model = build_model(config["Architecture"])

    load_model(config, model)

    # create data ops
    transforms = []
    for op in config["Eval"]["dataset"]["transforms"]:
        op_name = list(op)[0]
        if "Label" in op_name:
            continue
        elif op_name in ["RecResizeImg"]:
            op[op_name]["infer_mode"] = True
        elif op_name == "KeepKeys":
            if config["Architecture"]["algorithm"] == "SRN":
                op[op_name]["keep_keys"] = [
                    "image",
                    "encoder_word_pos",
                    "gsrm_word_pos",
                    "gsrm_slf_attn_bias1",
                    "gsrm_slf_attn_bias2",
                ]
            elif config["Architecture"]["algorithm"] == "SAR":
                op[op_name]["keep_keys"] = ["image", "valid_ratio"]
            elif config["Architecture"]["algorithm"] == "RobustScanner":
                op[op_name]["keep_keys"] = ["image", "valid_ratio", "word_positons"]
            else:
                op[op_name]["keep_keys"] = ["image"]
        transforms.append(op)
    global_config["infer_mode"] = True
    ops = create_operators(transforms, global_config)

    model.eval()
    return config, model, ops, post_process_class


# Carry out text recognition using PaddleOCR
def rec_ppocr(out_path, reg_yml, img_path, out_txt_name, ft_model_path=None):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    # Paddle OCR Configs
    recConfig, recModel, recOPS, recPost = init_rec(reg_yml, ft_model_path)

    images = glob.glob(os.path.join(img_path, "*"))  # load images

    with open(os.path.join(out_path, f"{out_txt_name}.txt"), mode="w") as out_f:
        for idx, img_name in enumerate(tqdm(images)):
            with open(img_name, "rb") as f:
                img = f.read()
                data = {"image": img}
            batch = transform(data, recOPS)
            images = np.expand_dims(batch[0], axis=0)
            images = paddle.to_tensor(images)
            preds = recModel(images)

            rec_res = recPost(preds)
            formatted_res = rec_res[0][0].replace(" ", "")
            out_f.write(f"{os.path.basename(img_name)}\t{formatted_res}\n")


# Chaining text detection and recognition end-to-end using PaddleOCR
def spot_ppocr(
    tmp_folder_path,
    out_path,
    det_yml,
    reg_yml,
    img_path,
    out_txt_name,
    det_ft_model_path=None,
    rec_ft_model_path=None,
):
    if os.path.isdir(tmp_folder_path):
        shutil.rmtree(tmp_folder_path)

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    # Paddle OCR Configs
    detConfig, detModel, detOPS, detPost = init_det(det_yml, det_ft_model_path)
    recConfig, recModel, recOPS, recPost = init_rec(reg_yml, rec_ft_model_path)

    images = glob.glob(os.path.join(img_path, "*"))  # load images

    output_dict = defaultdict(list)

    with open(os.path.join(out_path, f"{out_txt_name}.txt"), mode="w") as out_f:
        for idx, img_name in enumerate(tqdm(images)):
            img_id = os.path.basename(img_name)

            # -------------------------------- STAGE I DETECTION ------------------------------
            with open(img_name, "rb") as f:
                img = f.read()
                data = {"image": img}

            batch = transform(data, detOPS)
            images = np.expand_dims(batch[0], axis=0)
            shape_list = np.expand_dims(batch[1], axis=0)
            images = paddle.to_tensor(images)

            # forward & post process
            preds = detModel(images)
            det_res = detPost(preds, shape_list)

            # parser boxes if post_result is dict
            if isinstance(det_res, dict):
                for k in det_res.keys():
                    boxes = det_res[k][0]["points"]
            else:
                boxes = det_res[0]["points"]

            # Crop to patches
            ori_img = cv2.imread(img_name)
            h, w, c = ori_img.shape
            if not os.path.exists(tmp_folder_path):
                os.mkdir(tmp_folder_path)

            for i, box in enumerate(boxes):
                pt = box.tolist()
                try:
                    xs = [x[0] for x in pt]
                    xs = np.clip(np.array(xs), a_min=0, a_max=w).tolist()
                    ys = [x[1] for x in pt]
                    ys = np.clip(np.array(ys), a_min=0, a_max=h).tolist()
                    minx = min(xs)
                    miny = min(ys)
                    maxx = max(xs)
                    maxy = max(ys)

                    if (maxx - minx) <= 0 or (maxy - miny) <= 0:
                        print(
                            "BBOX error occured when cropping image"
                            f" {os.path.basename(img_name)} of {i}-th box"
                        )
                        print(f"Processed BBOX {minx} {miny} {maxx} {maxy}")
                        print(f"Ori BBOX {pt}")
                        continue

                    crop = ori_img[miny:maxy, minx:maxx]
                    tmp_img_name = os.path.join(
                        "{}/{}_{}.jpg".format(
                            tmp_folder_path,
                            img_name.split("/")[-1].split(".")[0],
                            str(i),
                        )
                    )
                    cv2.imwrite(tmp_img_name, crop)
                except Exception as e:
                    print(
                        f"Error {e} occured when cropping image"
                        f" {os.path.basename(img_name)} of {i}-th box"
                    )
                    continue

            boxes_len = len(boxes)
            tmp_folder_len = len(os.listdir(tmp_folder_path))
            if boxes_len == 0 or tmp_folder_len == 0:
                print(
                    f"No output for {os.path.basename(img_name)}, bbox_len:"
                    f" {boxes_len}, tmp_len: {tmp_folder_len}"
                )
                shutil.rmtree(tmp_folder_path)
                continue

            if boxes_len != tmp_folder_len:
                print(
                    f"mismatch bbox_len {boxes_len} and tmp_len {tmp_folder_len} for"
                    f" image {os.path.basename(img_name)}"
                )
                shutil.rmtree(tmp_folder_path)
                continue

            # ------------------------- Stage II Recognition ------------------------------
            for file in get_image_file_list(tmp_folder_path):
                with open(file, "rb") as f:
                    img = f.read()
                    data = {"image": img}
                batch = transform(data, recOPS)
                images = np.expand_dims(batch[0], axis=0)
                images = paddle.to_tensor(images)
                preds = recModel(images)

                rec_res = recPost(preds)
                formatted_res = rec_res[0][0].replace(" ", "")

                b_id = int(file.split("_")[-1].replace(".jpg", ""))
                crt_box = boxes[b_id]

                output_dict[img_id].append(
                    {
                        "points": crt_box.tolist(),
                        "transcription": formatted_res,
                    }
                )

            shutil.rmtree(tmp_folder_path)

            out_f.write(f"{img_id}\t{json.dumps(output_dict[img_id])}\n")

## Text Detection Using Pre-trained EAST


In [7]:
reset_dygraph()

In [8]:
out_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/east_output"
)
vis_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/east_output/vis"
)
det_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_east/det_mv3_east.yml"
)
img_path = "/content/drive/My Drive/CCPD2019/val"
out_txt_name = "east_output"

det_ppocr(out_path, vis_path, det_yml, img_path, out_txt_name)

[2023/05/11 10:05:52] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_east/det_mv3_east_v2.0_train/best_accuracy


100%|██████████| 20/20 [00:32<00:00,  1.63s/it]


## Text Detection Evaluation


In [9]:
# Let's evaluate the HMean for them
eval_config = {
    "IOU_CONSTRAINT": 0.5,
    "AREA_PRECISION_CONSTRAINT": 0.5,
    "WORD_SPOTTING": False,
}

gt_path = "/content/drive/My Drive/CCPD2019/val.txt"
det_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/east_output/east_output.txt"
)
resDict = det_eval(gt_path, det_path, eval_config)
precision, recall, hmean = (
    resDict["method"]["precision"],
    resDict["method"]["recall"],
    resDict["method"]["hmean"],
)

print("---Overall Metric---")
print(
    f"Precision: {round(precision, 2)}, Recall: {round(recall, 2)}, HMean:"
    f" {round(hmean, 2)}\n"
)

---Overall Metric---
Precision: 0.06, Recall: 0.25, HMean: 0.1



## Text Recognition Using Pre-trained CRNN


In [10]:
out_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/crnn_output"
)
reg_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_crnn/rec_mv3_none_bilstm_ctc.yml"
)
img_path = "/content/drive/My Drive/CCPD2019/val_crop"
out_txt_name = "crnn_output"

rec_ppocr(out_path, reg_yml, img_path, out_txt_name)

[2023/05/11 10:06:25] ppocr WARNING: The shape of model params head.fc.bias [64] not matched with loaded params head.fc.bias [37] !
[2023/05/11 10:06:25] ppocr WARNING: The shape of model params head.fc.weight [192, 64] not matched with loaded params head.fc.weight [192, 37] !
[2023/05/11 10:06:25] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_crnn/rec_mv3_none_bilstm_ctc_v2.0_train/best_accuracy


100%|██████████| 20/20 [00:05<00:00,  3.96it/s]


## Text Recognition Evaluation


In [11]:
# Let's evaluate the Accuracy for them
gt_path = "/content/drive/My Drive/CCPD2019/val_rec.txt"
det_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/crnn_output/crnn_output.txt"
)
gt, pred = data_prep_rec_eval(gt_path, det_path)
total_crw, total_crw_ci = total_accuracy(gt, pred)
total_edit, total_edit_ci = total_edit_distance(gt, pred)

print(f"Total Correctly Recognized Words: {total_crw}")
print(f"Total Correctly Recognized Words (Case Insensitive): {total_crw_ci}")
print(f"Accuracy: {round(total_crw/len(gt)*100, 2)} %")
print(f"Accuracy (Case Insensitive): {round(total_crw_ci/len(gt)*100, 2)} %")
print(f"Total Edit Distance: {total_edit}")
print(f"Total Edit Distance (Case Insensitive): {total_edit_ci}")

Total Correctly Recognized Words: 0
Total Correctly Recognized Words (Case Insensitive): 0
Accuracy: 0.0 %
Accuracy (Case Insensitive): 0.0 %
Total Edit Distance: 352
Total Edit Distance (Case Insensitive): 346


## Text Spotting Using Pre-trained EAST + CRNN


In [12]:
reset_dygraph()

In [13]:
tmp_folder_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/east_crnn_output/tmp"
)
out_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/east_crnn_output"
)
det_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_east/det_mv3_east.yml"
)
reg_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_crnn/rec_mv3_none_bilstm_ctc.yml"
)
img_path = "/content/drive/My Drive/CCPD2019/val"
out_txt_name = "east_crnn_output"

spot_ppocr(tmp_folder_path, out_path, det_yml, reg_yml, img_path, out_txt_name)

[2023/05/11 10:06:31] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_east/det_mv3_east_v2.0_train/best_accuracy
[2023/05/11 10:06:31] ppocr WARNING: The shape of model params head.fc.bias [64] not matched with loaded params head.fc.bias [37] !
[2023/05/11 10:06:31] ppocr WARNING: The shape of model params head.fc.weight [192, 64] not matched with loaded params head.fc.weight [192, 37] !
[2023/05/11 10:06:31] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_crnn/rec_mv3_none_bilstm_ctc_v2.0_train/best_accuracy


100%|██████████| 20/20 [00:26<00:00,  1.35s/it]


## Text Spotting Evaluation


In [14]:
# Let's evaluate the HMean for them
eval_config = {
    "IOU_CONSTRAINT": 0.5,
    "AREA_PRECISION_CONSTRAINT": 0.5,
    "WORD_SPOTTING": True,
}

gt_path = "/content/drive/My Drive/CCPD2019/val.txt"
det_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/pretrained_results/east_crnn_output/east_crnn_output.txt"
)
resDict = det_eval(gt_path, det_path, eval_config)
precision, recall, hmean = (
    resDict["method"]["precision"],
    resDict["method"]["recall"],
    resDict["method"]["hmean"],
)

print("---Overall Metric---")
print(
    f"Precision: {round(precision, 2)}, Recall: {round(recall, 2)}, HMean:"
    f" {round(hmean, 2)}\n"
)

---Overall Metric---
Precision: 0.0, Recall: 0.0, HMean: 0



## Fine-tuning EAST


In [15]:
# Since we cant use GPU for ppocr here, so I set Global.use_gpu=False
# Please set Global.use_gpu=True if you are running on a machine with GPU
# Fine-tuning for 100 epochs took 4 mins 29 secs on V100

# !python tools/train.py -c "/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_east/ft_det_mv3_east.yml" \
# -o Global.pretrained_model="/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_east/det_mv3_east_v2.0_train/best_accuracy" Global.use_gpu=False

## Fine-tuning CRNN


In [16]:
# Since we cant use GPU for ppocr here, so I set Global.use_gpu=False
# Please set Global.use_gpu=True if you are running on a machine with GPU
# Fine-tuning for 200 epochs took 5 mins 47 secs on V100

# !python tools/train.py -c "/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_crnn/ft_rec_mv3_none_bilstm_ctc.yml" \
# -o Global.pretrained_model="/content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/ppocr_crnn/rec_mv3_none_bilstm_ctc_v2.0_train/best_accuracy" Global.use_gpu=False

## Evaluate Fine-tuned EAST


In [17]:
reset_dygraph()

In [18]:
out_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/east_output"
)
vis_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/east_output/vis"
)
det_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_east/ft_det_mv3_east.yml"
)
img_path = "/content/drive/My Drive/CCPD2019/val"
out_txt_name = "east_output"
ft_model_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/fine_tuned_ccpd/east/best_accuracy"
)

det_ppocr(out_path, vis_path, det_yml, img_path, out_txt_name, ft_model_path)

[2023/05/11 10:06:59] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/fine_tuned_ccpd/east/best_accuracy


100%|██████████| 20/20 [00:24<00:00,  1.24s/it]


In [19]:
# Let's evaluate the HMean for them
eval_config = {
    "IOU_CONSTRAINT": 0.5,
    "AREA_PRECISION_CONSTRAINT": 0.5,
    "WORD_SPOTTING": False,
}

gt_path = "/content/drive/My Drive/CCPD2019/val.txt"
det_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/east_output/east_output.txt"
)
resDict = det_eval(gt_path, det_path, eval_config)
precision, recall, hmean = (
    resDict["method"]["precision"],
    resDict["method"]["recall"],
    resDict["method"]["hmean"],
)

print("---Overall Metric---")
print(
    f"Precision: {round(precision, 2)}, Recall: {round(recall, 2)}, HMean:"
    f" {round(hmean, 2)}\n"
)

---Overall Metric---
Precision: 0.91, Recall: 1.0, HMean: 0.95



## Evaluate Fine-tuned CRNN


In [20]:
out_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/crnn_output"
)
reg_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_crnn/ft_rec_mv3_none_bilstm_ctc.yml"
)
img_path = "/content/drive/My Drive/CCPD2019/val_crop"
out_txt_name = "crnn_output"
ft_model_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/fine_tuned_ccpd/crnn/best_accuracy"
)

rec_ppocr(out_path, reg_yml, img_path, out_txt_name, ft_model_path)

[2023/05/11 10:07:25] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/fine_tuned_ccpd/crnn/best_accuracy


100%|██████████| 20/20 [00:00<00:00, 25.19it/s]


In [21]:
# Let's evaluate the Accuracy for them
gt_path = "/content/drive/My Drive/CCPD2019/val_rec.txt"
det_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/crnn_output/crnn_output.txt"
)
gt, pred = data_prep_rec_eval(gt_path, det_path)
total_crw, total_crw_ci = total_accuracy(gt, pred)
total_edit, total_edit_ci = total_edit_distance(gt, pred)

print(f"Total Correctly Recognized Words: {total_crw}")
print(f"Total Correctly Recognized Words (Case Insensitive): {total_crw_ci}")
print(f"Accuracy: {round(total_crw/len(gt)*100, 2)} %")
print(f"Accuracy (Case Insensitive): {round(total_crw_ci/len(gt)*100, 2)} %")
print(f"Total Edit Distance: {total_edit}")
print(f"Total Edit Distance (Case Insensitive): {total_edit_ci}")

Total Correctly Recognized Words: 16
Total Correctly Recognized Words (Case Insensitive): 16
Accuracy: 80.0 %
Accuracy (Case Insensitive): 80.0 %
Total Edit Distance: 8
Total Edit Distance (Case Insensitive): 8


## Evaluate Fine-tuned EAST + CRNN


In [22]:
reset_dygraph()

In [23]:
tmp_folder_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/east_crnn_output/tmp"
)
out_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/east_crnn_output"
)
det_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_east/ft_det_mv3_east.yml"
)
reg_yml = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/ppocr_crnn/ft_rec_mv3_none_bilstm_ctc.yml"
)
img_path = "/content/drive/My Drive/CCPD2019/val"
out_txt_name = "east_crnn_output"
det_ft_model_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/fine_tuned_ccpd/east/best_accuracy"
)
rec_ft_model_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/fine_tuned_ccpd/crnn/best_accuracy"
)

spot_ppocr(
    tmp_folder_path,
    out_path,
    det_yml,
    reg_yml,
    img_path,
    out_txt_name,
    det_ft_model_path,
    rec_ft_model_path,
)

[2023/05/11 10:07:26] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/fine_tuned_ccpd/east/best_accuracy
[2023/05/11 10:07:27] ppocr INFO: load pretrain successful from /content/drive/My Drive/Colab Notebooks/Chapter 4/License_Plate_Recognition/fine_tuned_ccpd/crnn/best_accuracy


100%|██████████| 20/20 [00:24<00:00,  1.20s/it]


In [24]:
# Let's evaluate the HMean for them
eval_config = {
    "IOU_CONSTRAINT": 0.5,
    "AREA_PRECISION_CONSTRAINT": 0.5,
    "WORD_SPOTTING": True,
}

gt_path = "/content/drive/My Drive/CCPD2019/val.txt"
det_path = (
    "/content/drive/My Drive/Colab Notebooks/Chapter 4/"
    "License_Plate_Recognition/finetuned_results/east_crnn_output/east_crnn_output.txt"
)
resDict = det_eval(gt_path, det_path, eval_config)
precision, recall, hmean = (
    resDict["method"]["precision"],
    resDict["method"]["recall"],
    resDict["method"]["hmean"],
)

print("---Overall Metric---")
print(
    f"Precision: {round(precision, 2)}, Recall: {round(recall, 2)}, HMean:"
    f" {round(hmean, 2)}\n"
)

---Overall Metric---
Precision: 0.45, Recall: 0.5, HMean: 0.48

